In [139]:
from datasets import load_dataset, Dataset
import pandas as pd
import json

In [140]:
# Load original job details from JSON
job_details = load_dataset("json", data_files="./data/job_details.json")["train"]
job_details

Dataset({
    features: ['title', 'company', 'image_url', 'salary', 'industry', 'position', 'location', 'requirements', 'jd'],
    num_rows: 403
})

In [141]:
job_details = job_details.flatten()
job_details_df = pd.DataFrame(job_details)
job_details_df = job_details_df.drop_duplicates()
job_details = Dataset.from_pandas(job_details_df)
job_details

Dataset({
    features: ['title', 'company', 'image_url', 'salary', 'industry', 'position', 'location', 'requirements.context', 'requirements.degree', 'requirements.experience', 'jd', '__index_level_0__'],
    num_rows: 401
})

In [142]:
# Load preprocessed job data from CSV
preprocessed_data = load_dataset("csv", data_files="./data/job_preprocessed.csv")[
    "train"
]
preprocessed_data

Dataset({
    features: ['title', 'company', 'image_url', 'salary', 'industry', 'position', 'location', 'jd', 'degree', 'experience', 'context', 'skills'],
    num_rows: 401
})

In [143]:
job_details = job_details.add_column("skills", preprocessed_data["skills"])
job_details.features

{'title': Value(dtype='string', id=None),
 'company': Value(dtype='string', id=None),
 'image_url': Value(dtype='string', id=None),
 'salary': Value(dtype='string', id=None),
 'industry': Value(dtype='string', id=None),
 'position': Value(dtype='string', id=None),
 'location': Value(dtype='string', id=None),
 'requirements.context': Value(dtype='string', id=None),
 'requirements.degree': Value(dtype='string', id=None),
 'requirements.experience': Value(dtype='string', id=None),
 'jd': Value(dtype='string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None),
 'skills': Value(dtype='string', id=None)}

In [144]:
texts = []

for row in preprocessed_data:
    skills = row["skills"]
    context = row["context"]
    jd = row["jd"]
    combined_text = f"[JD] {jd} \n[Context] {context} \n[Skills] {skills}"
    texts.append(combined_text)

In [145]:
job_details = job_details.add_column("combined_text", texts)

In [146]:
job_details.features

{'title': Value(dtype='string', id=None),
 'company': Value(dtype='string', id=None),
 'image_url': Value(dtype='string', id=None),
 'salary': Value(dtype='string', id=None),
 'industry': Value(dtype='string', id=None),
 'position': Value(dtype='string', id=None),
 'location': Value(dtype='string', id=None),
 'requirements.context': Value(dtype='string', id=None),
 'requirements.degree': Value(dtype='string', id=None),
 'requirements.experience': Value(dtype='string', id=None),
 'jd': Value(dtype='string', id=None),
 '__index_level_0__': Value(dtype='int64', id=None),
 'skills': Value(dtype='string', id=None),
 'combined_text': Value(dtype='string', id=None)}

In [147]:
columns_mapping = {
    "company": "companyName",
    "image_url": "imageURL",
    "salary": "salaryRange",
    "requirements.context": "requirementContext",
    "requirements.degree": "requiredDegree",
    "requirements.experience": "requiredExperience",
    "jd": "jobDescription",
    "skills": "skillsRequired",
}

In [148]:
job_details = job_details.rename_columns(columns_mapping)
job_details = job_details.remove_columns(["__index_level_0__"])
job_details

Dataset({
    features: ['title', 'companyName', 'imageURL', 'salaryRange', 'industry', 'position', 'location', 'requirementContext', 'requiredDegree', 'requiredExperience', 'jobDescription', 'skillsRequired', 'combined_text'],
    num_rows: 401
})

In [149]:
job_details["combined_text"][:3]

["[JD] nhiệm vụ quản lý  dẫn dắt đội ngũ phát triển ứng dụng  trang web \n hướng dẫn  đào tạo hỗ trợ kỹ thuật \n ước lượng khối lượng công việc chi phí cho dự án \n giao tiếp đàm phán khách hàng nhiệm vụ kỹ thuật  dẫn dắt việc lập kế hoạch  thiết kế phát triển hệ thống phần mềm hoặc phần cứng nhằm giúp công ty hoặc khách hàng đạt mục tiêu \n xây dựng kế hoạch duy trì hoạt động kinh doanh \n cung cấp hướng dẫn kỹ thuật đạo cụ thể cho thành viên cấp \n[Context] Tốt nghiệp Đại học chuyên ngành CNTT hoặc ngành liên quan Tiếng Nhật : JLPT N3 trở lên Tối thiểu 2 năm kinh nghiệm quản lý dự án vai trò BrSE , Trưởng Dự Án , hoặc Trưởng Kinh nghiệm phát triển phần mềm ngôn ngữ Java , .Net , Python , v.v . Kiến thức tốt hoặc kinh nghiệm thực tế về cơ sở dữ liệu SQL / non - SQL MS SQL , PostgreSQL , MongoDB , v.v . \n[Skills] ['giao tiếp', '.NET', 'sql', 'đàm phán', 'postgresql', 'quản lý dự án', 'python', 'Tốt nghiệp Đại học', '2 năm kinh nghiệm', 'mongodb', 'java']",
 '[JD] số lượng \n  01  mô t

In [150]:
job_details.to_json(
    "jobs.json",
    orient="records",  # Lưu mỗi hàng dưới dạng một đối tượng JSON riêng lẻ
    lines=False,  # Mỗi dòng là một JSON object
    force_ascii=False,  # Giữ nguyên các ký tự không phải ASCII (Unicode)
    indent=4,  # Tăng khả năng đọc với indent
)

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 55.20ba/s]


1535574

In [151]:
json_job_details = load_dataset("json", data_files="jobs.json")["train"]
json_job_details

Generating train split: 401 examples [00:00, 14513.66 examples/s]


Dataset({
    features: ['title', 'companyName', 'imageURL', 'salaryRange', 'industry', 'position', 'location', 'requirementContext', 'requiredDegree', 'requiredExperience', 'jobDescription', 'skillsRequired', 'combined_text'],
    num_rows: 401
})

In [152]:
json_job_details[:3]["title"]

['[Quận Tân Bình, HCM] Trưởng Nhóm Dự Án IT (Tiếng Nhật) (35Tr-52Tr) (24996)',
 'Nhân Viên M&amp;E (Mảng Nước)',
 'Talent Acquisition - HCMC']